# Sales Forecasting Using TabTransformer (Notebook Entry Point)

This notebook is the main entry point for the project submission.

It reproduces the full pipeline that the CLI runs:
1) Prepare data (feature engineering + processed parquet/metadata)
2) Train model (linreg / xgb / tabtx)
3) Evaluate model (metrics + plots)
4) Predict on input CSV and write preds.csv

Outputs are written under:
- data/processed/
- artifacts/<model_name>/


In [ ]:
# imports and path setup
from __future__ import annotations
from pathlib import Path
import pandas as pd
from src import utils, data, train, evaluate
from src.metrics import metrics_table, MetricsReport


In [2]:
# folder check
ROOT = Path(".").resolve()
RAW_DIR = ROOT / "data" / "raw"
PROCESSED_DIR = ROOT / "data" / "processed"
ARTIFACTS_DIR = ROOT / "artifacts"
RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)
RAW_DIR, PROCESSED_DIR, ARTIFACTS_DIR


(WindowsPath('C:/Users/Chris/Desktop/To Submit/PythonEngineering/FinalProject/local_FinalProject/Sales-Forcasting-TabTransformer/data/raw'),
 WindowsPath('C:/Users/Chris/Desktop/To Submit/PythonEngineering/FinalProject/local_FinalProject/Sales-Forcasting-TabTransformer/data/processed'),
 WindowsPath('C:/Users/Chris/Desktop/To Submit/PythonEngineering/FinalProject/local_FinalProject/Sales-Forcasting-TabTransformer/artifacts'))

In [ ]:
# load config + notebook overrides
config_path = ROOT / "config" / "default.yaml"
config = utils.load_yaml(config_path)

# can remove for full training
config.setdefault("training", {})
config["training"].update({
    "epochs": 2,        
    "batch_size": 32,
    "patience": 1,
    "seed": 123,
    "device": "cpu",
    "amp": False,
})

config.setdefault("splits", {})
config["splits"].update({
    "strategy": "ratios",
    "train_ratio": 0.6,
    "val_ratio": 0.2,
    "test_ratio": 0.2,
})

# ensure paths point to repo-local folders
config.setdefault("paths", {})
config["paths"].update({
    "processed_dir": str(PROCESSED_DIR),
    "artifacts_dir": str(ARTIFACTS_DIR),
})

config


{'paths': {'raw_csv': 'data/raw/train.csv',
  'train_csv': 'data/raw/train.csv',
  'test_csv': 'data/raw/test.csv',
  'store_csv': 'data/raw/store.csv',
  'processed_dir': 'C:\\Users\\Chris\\Desktop\\To Submit\\PythonEngineering\\FinalProject\\local_FinalProject\\Sales-Forcasting-TabTransformer\\data\\processed',
  'processed_filename': 'sales_processed.parquet',
  'metadata_filename': 'metadata.json',
  'artifacts_dir': 'C:\\Users\\Chris\\Desktop\\To Submit\\PythonEngineering\\FinalProject\\local_FinalProject\\Sales-Forcasting-TabTransformer\\artifacts'},
 'features': {'target': 'Sales',
  'horizon': 1,
  'categorical': ['Store',
   'DayOfWeek',
   'Promo',
   'StateHoliday',
   'SchoolHoliday'],
  'numeric': ['Customers',
   'Sales',
   'CompetitionDistance',
   'Sales_lag_1',
   'Sales_lag_7',
   'Sales_lag_14',
   'Sales_roll7',
   'Sales_roll14',
   'year',
   'month',
   'day',
   'weekofyear',
   'is_weekend']},
 'splits': {'strategy': 'ratios',
  'train_ratio': 0.6,
  'val_rati

In [ ]:
# CSV creation - users can run without Kaggle
records = []
base_date = pd.Timestamp("2024-01-01")

for store in range(1, 4):
    for offset in range(60):  
        current_date = base_date + pd.Timedelta(days=offset)
        records.append({
            "Store": store,
            "DayOfWeek": int(current_date.dayofweek + 1),
            "Date": current_date.strftime("%Y-%m-%d"),
            "Promo": offset % 2,
            "StateHoliday": "0",
            "SchoolHoliday": 0,
            "Customers": 120 + store * 5 + offset * 2,
            "Sales": 1000 + store * 25 + offset * 8,
            "CompetitionDistance": 400.0 + store * 10,
        })

synthetic_df = pd.DataFrame(records)
synthetic_df.head()


,Store,DayOfWeek,Date,Promo,StateHoliday,SchoolHoliday,Customers,Sales,CompetitionDistance
0,1,1,2024-01-01,0,0,0,125,1025,410.0
1,1,2,2024-01-02,1,0,0,127,1033,410.0
2,1,3,2024-01-03,0,0,0,129,1041,410.0
3,1,4,2024-01-04,1,0,0,131,1049,410.0
4,1,5,2024-01-05,0,0,0,133,1057,410.0


In [5]:
# write CSV and point config 
synth_path = RAW_DIR / "notebook_synthetic.csv"
synthetic_df.to_csv(synth_path, index=False)

config["paths"]["raw_csv"] = str(synth_path)
synth_path


WindowsPath('C:/Users/Chris/Desktop/To Submit/PythonEngineering/FinalProject/local_FinalProject/Sales-Forcasting-TabTransformer/data/raw/notebook_synthetic.csv')

In [ ]:
# prepare data - feature engineering, processed parquet, metadata
X, y, metadata = data.load_sales_data(synth_path, config, save=True)
print("Prepared dataset:")
print("X shape:", X.shape)
print("y shape:", y.shape)
print("Metadata keys:", sorted(list(metadata.keys())))

# example data
X.head(), y.head()


2025-12-17 23:07:06,835 | INFO | src.data | Saved processed dataset to C:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\data\processed\sales_processed.parquet


Prepared dataset:
X shape: (177, 18)
y shape: (177,)
Metadata keys: ['categorical_features', 'dates', 'feature_columns', 'horizon', 'numeric_features', 'stores', 'target']


(  Store DayOfWeek Promo StateHoliday SchoolHoliday  Customers  Sales  \
 0     1         1     0            0             0        125   1025   
 1     1         2     1            0             0        127   1033   
 2     1         3     0            0             0        129   1041   
 3     1         4     1            0             0        131   1049   
 4     1         5     0            0             0        133   1057   
 
    CompetitionDistance  Sales_lag_1  Sales_lag_7  Sales_lag_14  Sales_roll7  \
 0                410.0       1025.0       1025.0        1025.0       1025.0   
 1                410.0       1025.0       1025.0        1025.0       1025.0   
 2                410.0       1033.0       1025.0        1025.0       1029.0   
 3                410.0       1041.0       1025.0        1025.0       1033.0   
 4                410.0       1049.0       1025.0        1025.0       1037.0   
 
    Sales_roll14  year  month  day  weekofyear  is_weekend  
 0        1025.0 

In [8]:
# train linear regression - baseline
linreg_result = train.train_pipeline(config, "linreg")
linreg_result


2025-12-17 23:07:11,401 | INFO | src.data | Loaded processed dataset from C:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\data\processed\sales_processed.parquet
2025-12-17 23:07:11,479 | INFO | src.train | Train split metrics:
### train Metrics
| Metric | Value |
| --- | --- |
| MAE | 0.0000 |
| RMSE | 0.0000 |
| R2 | 1.0000 |
| MAPE | 0.0000 |
2025-12-17 23:07:11,488 | INFO | src.train | Val split metrics:
### val Metrics
| Metric | Value |
| --- | --- |
| MAE | 0.0000 |
| RMSE | 0.0000 |
| R2 | 1.0000 |
| MAPE | 0.0000 |
2025-12-17 23:07:11,501 | INFO | src.train | Test split metrics:
### test Metrics
| Metric | Value |
| --- | --- |
| MAE | 0.0000 |
| RMSE | 0.0000 |
| R2 | 1.0000 |
| MAPE | 0.0000 |
2025-12-17 23:07:11,517 | INFO | src.train | Training complete. Artifacts stored in C:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\artifacts\linreg
2025

{'artifacts': 'C:\\Users\\Chris\\Desktop\\To Submit\\PythonEngineering\\FinalProject\\local_FinalProject\\Sales-Forcasting-TabTransformer\\artifacts\\linreg',
 'metrics': {'train': {'MAE': 1.0296166435165226e-13,
   'RMSE': 1.530057909145519e-13,
   'R2': 1.0,
   'MAPE': 8.31513345406509e-15},
  'val': {'MAE': 2.468628476240805e-13,
   'RMSE': 2.549367812122855e-13,
   'R2': 1.0,
   'MAPE': 1.7801533498103764e-14},
  'test': {'MAE': 3.9158799659667746e-13,
   'RMSE': 4.0461442085989314e-13,
   'R2': 1.0,
   'MAPE': 2.638709813288198e-14}}}

In [9]:
# train XGBoost - baseline
try:
    import xgboost  # noqa: F401
    xgb_result = train.train_pipeline(config, "xgb")
    xgb_result
except Exception as exc:
    print("Skipping XGBoost training (missing xgboost or error):", exc)
    xgb_result = None


2025-12-17 23:09:31,709 | INFO | src.data | Loaded processed dataset from C:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\data\processed\sales_processed.parquet
2025-12-17 23:09:32,623 | INFO | src.train | Train split metrics:
### train Metrics
| Metric | Value |
| --- | --- |
| MAE | 0.0018 |
| RMSE | 0.0023 |
| R2 | 1.0000 |
| MAPE | 0.0001 |
2025-12-17 23:09:32,643 | INFO | src.train | Val split metrics:
### val Metrics
| Metric | Value |
| --- | --- |
| MAE | 33.2048 |
| RMSE | 43.1531 |
| R2 | -0.7353 |
| MAPE | 2.3551 |
2025-12-17 23:09:32,658 | INFO | src.train | Test split metrics:
### test Metrics
| Metric | Value |
| --- | --- |
| MAE | 125.2503 |
| RMSE | 130.5106 |
| R2 | -13.4429 |
| MAPE | 8.4213 |
2025-12-17 23:09:32,689 | INFO | src.train | Training complete. Artifacts stored in C:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\artifacts\xg

In [10]:
# train TabTransformer
tabtx_result = train.train_pipeline(config, "tabtx")
tabtx_result


2025-12-17 23:09:35,290 | INFO | src.data | Loaded processed dataset from C:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\data\processed\sales_processed.parquet
c:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\src\models\tabtransformer.py:243: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=bool(training_config.get("amp", False)))
c:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\src\models\tabtransformer.py:255: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=scaler.is_enabled()):
2025-12-17 23:09:40,384 | INFO | src.train | Train split metrics:
### train M

{'artifacts': 'C:\\Users\\Chris\\Desktop\\To Submit\\PythonEngineering\\FinalProject\\local_FinalProject\\Sales-Forcasting-TabTransformer\\artifacts\\tabtx',
 'metrics': {'train': {'MAE': 1191.3483849224056,
   'RMSE': 1194.2878388329668,
   'R2': -202.42672776345887,
   'MAPE': 99.68259578907653},
  'val': {'MAE': 1380.2350244522095,
   'RMSE': 1380.6239753505224,
   'R2': -1775.2544369144703,
   'MAPE': 99.7298529438007},
  'test': {'MAE': 1474.2629908919334,
   'RMSE': 1474.663075574742,
   'R2': -1842.94956455361,
   'MAPE': 99.7470097528111}}}

In [ ]:
# model evaluation by test split and show metrics table
results = {}

for model_name in ["linreg", "xgb", "tabtx"]:
    if model_name == "xgb" and xgb_result is None:
        continue
    metric_values = evaluate.evaluate_pipeline(config, model_name, split="test")
    results[model_name] = metric_values

    print(f"\n=== {model_name.upper()} (TEST) ===")
    print(metrics_table(metric_values))
    try:
        display(MetricsReport(metric_values, label=f"{model_name.upper()} Test"))
    except Exception:
        pass

results


2025-12-17 23:09:47,829 | INFO | src.data | Loaded processed dataset from C:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\data\processed\sales_processed.parquet
2025-12-17 23:09:51,644 | INFO | src.evaluate | Evaluation metrics (test):
### test Metrics
| Metric | Value |
| --- | --- |
| MAE | 0.0000 |
| RMSE | 0.0000 |
| R2 | 1.0000 |
| MAPE | 0.0000 |



=== LINREG (TEST) ===
| Metric | Value |
| --- | --- |
| MAE | 0.0000 |
| RMSE | 0.0000 |
| R2 | 1.0000 |
| MAPE | 0.0000 |


MetricsReport(values={'MAE': 3.9158799659667746e-13, 'RMSE': 4.0461442085989314e-13, 'R2': 1.0, 'MAPE': 2.638709813288198e-14}, label='LINREG Test')

2025-12-17 23:09:51,657 | INFO | src.data | Loaded processed dataset from C:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\data\processed\sales_processed.parquet
2025-12-17 23:09:52,237 | INFO | src.evaluate | Evaluation metrics (test):
### test Metrics
| Metric | Value |
| --- | --- |
| MAE | 125.2503 |
| RMSE | 130.5106 |
| R2 | -13.4429 |
| MAPE | 8.4213 |



=== XGB (TEST) ===
| Metric | Value |
| --- | --- |
| MAE | 125.2503 |
| RMSE | 130.5106 |
| R2 | -13.4429 |
| MAPE | 8.4213 |


MetricsReport(values={'MAE': 125.25029161241319, 'RMSE': 130.5106485922041, 'R2': -13.442930522293207, 'MAPE': 8.421305042729404}, label='XGB Test')

2025-12-17 23:09:52,252 | INFO | src.data | Loaded processed dataset from C:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\data\processed\sales_processed.parquet
c:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\src\models\tabtransformer.py:338: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user vi


=== TABTX (TEST) ===
| Metric | Value |
| --- | --- |
| MAE | 1474.2630 |
| RMSE | 1474.6631 |
| R2 | -1842.9496 |
| MAPE | 99.7470 |


MetricsReport(values={'MAE': 1474.2629908919334, 'RMSE': 1474.663075574742, 'R2': -1842.94956455361, 'MAPE': 99.7470097528111}, label='TABTX Test')

{'linreg': {'MAE': 3.9158799659667746e-13,
  'RMSE': 4.0461442085989314e-13,
  'R2': 1.0,
  'MAPE': 2.638709813288198e-14},
 'xgb': {'MAE': 125.25029161241319,
  'RMSE': 130.5106485922041,
  'R2': -13.442930522293207,
  'MAPE': 8.421305042729404},
 'tabtx': {'MAE': 1474.2629908919334,
  'RMSE': 1474.663075574742,
  'R2': -1842.94956455361,
  'MAPE': 99.7470097528111}}

In [12]:
# demonstrate MetricsReport usage (operator overloading & aggregation)
linreg_report = MetricsReport(results["linreg"], label="Linear Regression")
tabtx_report  = MetricsReport(results["tabtx"], label="TabTransformer")

# example: average metrics across two models
avg_report = (linreg_report + tabtx_report) / 2

print("Averaged Metrics (LinReg + TabTransformer):")
display(avg_report)

# access as dictionary if needed
avg_report.to_dict()


Averaged Metrics (LinReg + TabTransformer):


MetricsReport(values={'R2': -920.974782276805, 'MAE': 737.131495445967, 'RMSE': 737.3315377873712, 'MAPE': 49.87350487640556}, label='Linear Regression')

{'R2': -920.974782276805,
 'MAE': 737.131495445967,
 'RMSE': 737.3315377873712,
 'MAPE': 49.87350487640556}

In [ ]:
# predict on the csv and write preds to a file
output_preds_path = ARTIFACTS_DIR / "notebook_preds.csv"

# demonstrate prediction + file I/O using a single model - same API works for "xgb" and "tabtx".
PRED_MODEL = "linreg"

artifact_dir = utils.get_artifact_dir(config, PRED_MODEL)
model = evaluate.load_trained_model(PRED_MODEL, artifact_dir, config)

# load features 
X_full, y_full, metadata_full = data.load_sales_data(synth_path, config, save=False)

preds = model.predict(X_full)
preds_df = pd.DataFrame({"prediction": preds.values}, index=X_full.index)
preds_df.to_csv(output_preds_path, index=False)

print("Wrote predictions to:", output_preds_path)
preds_df.head()


Wrote predictions to: C:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\artifacts\notebook_preds.csv


,prediction
0,1040.396014
1,1046.311353
2,1052.226692
3,1058.142030
4,1064.057369


In [14]:
# demonstrate file I/O side effects (artifacts created by the pipeline)
from pathlib import Path

print("Files created under artifacts/linreg:")
artifact_dir = Path("artifacts/linreg")
if artifact_dir.exists():
    for f in sorted(artifact_dir.iterdir()):
        print(" -", f.name)
else:
    print(" (artifacts/linreg does not exist yet — train/eval linreg first)")

Files created under artifacts/linreg:
 - config_snapshot.json
 - metrics.json
 - metrics_test.json
 - model.joblib
 - pred_vs_actual.png
 - residual_hist.png


In [15]:
# outputs
processed_path = PROCESSED_DIR / config["paths"].get("processed_filename", "sales.parquet")
print("Processed parquet exists:", processed_path.exists(), "->", processed_path)

print("\nArtifacts directory contents:")
for p in sorted(ARTIFACTS_DIR.glob("*")):
    if p.is_dir():
        print("-", p.name, "->", [x.name for x in sorted(p.iterdir())][:10], "...")
    else:
        print("-", p.name)


Processed parquet exists: True -> C:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\data\processed\sales_processed.parquet

Artifacts directory contents:
- linreg -> ['config_snapshot.json', 'metrics.json', 'metrics_test.json', 'model.joblib', 'pred_vs_actual.png', 'residual_hist.png'] ...
- notebook_preds.csv
- tabtx -> ['config_snapshot.json', 'metrics.json', 'metrics_test.json', 'model.pt', 'pred_vs_actual.png', 'residual_hist.png'] ...
- xgb -> ['config_snapshot.json', 'metrics.json', 'metrics_test.json', 'model.joblib', 'pred_vs_actual.png', 'residual_hist.png'] ...


## Experiment Name Support (Artifact Isolation)

This section demonstrates that setting `experiment_name` in the YAML configuration
automatically changes where model artifacts are written. This allows multiple
experiments to coexist without overwriting results.


In [ ]:
print("Configured experiment_name:", config.get("experiment_name"))
artifact_dir = utils.get_artifact_dir(config, "linreg")
print("Resolved artifact directory:")
print(" ", artifact_dir)
print("\nDirectory contents:")
if artifact_dir.exists():
    for f in sorted(artifact_dir.iterdir()):
        print(" -", f.name)
else:
    print(" (artifact directory does not exist yet)")


Configured experiment_name: None
Resolved artifact directory:
  C:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\artifacts\linreg

Directory contents:
 - config_snapshot.json
 - metrics.json
 - metrics_test.json
 - model.joblib
 - pred_vs_actual.png
 - residual_hist.png


In [ ]:
# simulate a second experiment in-notebook
config_alt = dict(config)
config_alt["experiment_name"] = "notebook_experiment_alt"
alt_dir = utils.get_artifact_dir(config_alt, "linreg")
print("Alternate experiment_name:", config_alt["experiment_name"])
print("Alternate artifact dir:", alt_dir)


Alternate experiment_name: notebook_experiment_alt
Alternate artifact dir: C:\Users\Chris\Desktop\To Submit\PythonEngineering\FinalProject\local_FinalProject\Sales-Forcasting-TabTransformer\artifacts\linreg
